<a href="https://colab.research.google.com/github/Subina00/blockchain-project/blob/main/Copy_of_Assessment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
## Import statements
import hashlib
import time
from typing import List

# ------------------------- CUSTOM EXCEPTIONS -------------------------
class BlockchainError(Exception):
    """ Base for Blockchain errors."""
    pass


class InvalidBlockError(BlockchainError):
    """ The raised block when it fails validation checks."""
    pass

# ------------------------- BLOCK CLASS -------------------------

class Block:
  # Block instance attributes
    def __init__(self, index, previous_hash, timestamp, data, proof):

        self.index = index                # Position of the block in the blockchain
        self.previous_hash = previous_hash # Hash of the previous block to maintain linkage
        self.timestamp = timestamp        # Time of block creation
        self.data = data                  # Transaction data or user input
        self.proof = proof                # Proof of work value
        self.hash = self.calculate_hash() # Current block's hash

    def calculate_hash(self):

        """
        Concatenates block properties to a string and uses the SHA-256 algorithm to create a unique hash.
        This makes blocks immutable and integrity assured.
        """

        block_content = f"{self.index}{self.previous_hash}{self.timestamp}{self.data}{self.proof}"
        return hashlib.sha256(block_content.encode()).hexdigest()

# ---------------------- BLOCKCHAIN CLASS -----------------------

class Blockchain:

    def __init__(self, difficulty: int = 4) -> None: #Number of leading zeros required in the hash
        self.chain: List[Block] = [self.create_genesis_block()] # Blockchain starts with the genesis block
        self.difficulty = difficulty


    def create_genesis_block(self):
        """
        Generates the first block in the blockchain (genesis block) with fixed data.
        """
        return Block(0, "0", time.time(), "Genesis Block", 0)

    def get_latest_block(self):
        return self.chain[-1]


    def proof_of_work(self, block):
        """
        Mining role: updates proofs until the hash of the block fulfills difficulty requirements.
        Proof-of-Work is security through computational infeasibility of alteration.
        """
        block.proof = 0
        block.hash = block.calculate_hash()
      # Keep recalculating hash until it starts with '0000' (difficulty = 4)
        while not block.hash.startswith('0' * self.difficulty):
            block. proof += 1
            block.hash = block.calculate_hash()
        return block

    def add_block(self, new_block):
        """
         Connects a fresh block to blockchain by passing the Proof-of-Work.
         Modifies the previous hash in the block and adds it to the chain.

        """
        new_block.previous_hash = self.get_latest_block().hash
        mined_block = self.proof_of_work(new_block)
        self.chain.append(mined_block)

    def add_data(self, data):
        """
        Creates a new block containing user data, mines it using PoW, and adds it to the chain.
        """
        new_block = Block(
            index=len(self.chain),
            previous_hash=self.get_latest_block().hash,
            timestamp=time.time(),
            data=data,
            proof=0
        )
        self.add_block(new_block)

    def is_chain_valid(self):
        """
        Verifies the integrity of the whole block chain.
        Checks:
        1. Block hashes are accurate (no tampering of data).
        2. Previous hash field is matched with hash of previous block.
        3. All the blocks meet the Proof-of-Work requirement.
        """
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i - 1]

            # Recalculate hash and compare
            if current.hash != current.calculate_hash():
                print(f"Hash mismatch at block {current.index}")
                return False

            # Check linkage
            if current.previous_hash != previous.hash:
                print(f"Previous hash mismatch at block {current.index}")
                return False

            # Validate proof-of-work
            if not current.hash.startswith('0' * self.difficulty):
                print(f"Proof of work not satisfied at block {current.index}")
                return False

        return True

# ----------------------- MAIN EXECUTION ------------------------
# Example Usage
if __name__ == "__main__":
  # Create a new blockchain instance
    blockchain = Blockchain()

    # Mine and add first block with transaction data
    print("Mining block 1...")
    blockchain.add_data("Transaction data for Block 1")

    # Mine and add second block
    print("Mining block 2...")
    blockchain.add_data("Transaction data for Block 2")

    # Validate the blockchain
    print("\nBlockchain validity:", blockchain.is_chain_valid())

     # Display the entire blockchain content in a neat format
    print("\n--- Blockchain Contents ---")
    for block in blockchain.chain:
        print(f"Block {block.index}")
        print(f"Timestamp: {time.ctime(block.timestamp)}") # Convert timestamp to readable format
        print(f"Data: {block.data}")
        print(f"Hash: {block.hash}")
        print(f"Previous Hash: {block.previous_hash}")
        print(f"Proof: {block.proof}")
        print("-" * 50)



Mining block 1...
Mining block 2...

Blockchain validity: True

--- Blockchain Contents ---
Block 0
Timestamp: Sat Jul 19 06:16:37 2025
Data: Genesis Block
Hash: 442f8380691f9353b2a3d38854e0078caca64d26f657ebad7f2331d353c0c37f
Previous Hash: 0
Proof: 0
--------------------------------------------------
Block 1
Timestamp: Sat Jul 19 06:16:37 2025
Data: Transaction data for Block 1
Hash: 0000e6f9d9114542b8d79a5766f1b1477cbe8978a0633cfbdea151128b7a2144
Previous Hash: 442f8380691f9353b2a3d38854e0078caca64d26f657ebad7f2331d353c0c37f
Proof: 147598
--------------------------------------------------
Block 2
Timestamp: Sat Jul 19 06:16:37 2025
Data: Transaction data for Block 2
Hash: 0000173046825a53e76dbb81e68cff3caa74771f35eefd0dc7db0549360ef553
Previous Hash: 0000e6f9d9114542b8d79a5766f1b1477cbe8978a0633cfbdea151128b7a2144
Proof: 262917
--------------------------------------------------
